# Test the endpoint

In [131]:
import json
import pandas as pd
import numpy as np

In [132]:
import boto3
import sagemaker
from sagemaker import get_execution_role

client = boto3.client('sagemaker-runtime')

## Run Inference on deployed endpoint

In [133]:
df = pd.read_csv("data/mtsamples.csv")

idx = np.random.randint(len(df))
input_txt = df.loc[idx].transcription
category = df.loc[idx].medical_specialty
print(category)
print(input_txt)

 Obstetrics / Gynecology
PREOPERATIVE DIAGNOSIS:,  Ovarian cyst, persistent.,POSTOPERATIVE DIAGNOSIS: , Ovarian cyst.,ANESTHESIA:,  General,NAME OF OPERATION:,  Diagnostic laparoscopy and drainage of cyst.,PROCEDURE:,  The patient was taken to the operating room, prepped and draped in the usual manner, and adequate anesthesia was induced.  An infraumbilical incision was made, and Veress needle placed without difficulty.  Gas was entered into the abdomen at two liters.  The laparoscope was entered, and the abdomen was visualized.  The second puncture site was made, and the second trocar placed without difficulty.  The cyst was noted on the left, a 3-cm, ovarian cyst.  This was needled, and a hole cut in it with the scissors.  Hemostasis was intact.  Instruments were removed.  The patient was awakened and taken to the recovery room in good condition.


In [ ]:
endpoint_name = "31-2023-04-05-07-46-59-107"
content_type = "text/csv" # "application/json" #
accept = "application/json" #"text/csv"
# payload = "test"#json.dumps({"instances":["test"]})

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    # CustomAttributes=custom_attributes,
    ContentType=content_type,
    Accept=accept,
    Body=input_txt,
)

json.loads(response["Body"].read())


## Run batch inference (Transformer)
Use latest approved model from model registry

In [141]:
def get_latest_approved_model(model_package_group_name):
    sm_client = boto3.client('sagemaker')
    df = pd.DataFrame(sm_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"])
    # models are allways sorted newst to oldest
    return df.loc[df.ModelApprovalStatus == "Approved"].iloc[0].ModelPackageArn

In [142]:
session = sagemaker.Session()
default_bucket = session.default_bucket()

input_data_path = f"s3://{default_bucket}/data/test.csv"
output_data_path = f"s3://{default_bucket}/data/out"
CONTENT_TYPE_CSV = 'text/csv'

iam = boto3.client('iam')
role_arn = iam.get_role(RoleName=f'101436505502-sagemaker-exec')['Role']['Arn']

model_package_group_name="training-pipelineModelGroup"
model_package_arn = get_latest_approved_model(model_package_group_name)
# model_package_arn = "arn:aws:sagemaker:eu-west-3:101436505502:model-package/training-pipelinemodelgroup/33"
print(model_package_arn)
# load model
model = sagemaker.ModelPackage(role=role_arn, model_package_arn=model_package_arn,
                         sagemaker_session=session)

# create transformer model
transformer =model.transformer(
    instance_count = 1,
    instance_type = "ml.m5.large",#'ml.g4dn.xlarge', #
    strategy = 'SingleRecord',
    assemble_with = 'Line',
    output_path = output_data_path,
    accept = CONTENT_TYPE_CSV,
)

arn:aws:sagemaker:eu-west-3:101436505502:model-package/training-pipelinemodelgroup/36


INFO:sagemaker:Creating model with name: 36-2023-04-06-08-33-23-533


In [143]:
transformer.transform(data=input_data_path,
                      content_type=CONTENT_TYPE_CSV, split_type='Line',)
# transformer.wait()

INFO:sagemaker:Creating transform job with name: 36-2023-04-06-08-33-33-420


...................